### Ideen Use Case
- Gesetzbuch ZGB
- Kochbuch
- LLM Agent generiert Werbetexe, Social Media Posts

### FWO-Assistent

- Hilft bei der Erstellung von Texten für LinkedIn, Instagram, WhatsApp etc.

- Auf Basis von vergangenen Posts

In [1]:
import os
import langchain
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI


LLM_MODEL = "openai/gpt-oss-20b:free"
LLM_TEMPERATURE = 0.4
BASE_URL = "https://openrouter.ai/api/v1"
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
USER_PROMPT="Generiere mir einen Instagrampost"

c:\Users\amuel\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOpenAI(
    model=LLM_MODEL,
    temperature=LLM_TEMPERATURE,
    base_url=BASE_URL,
    api_key=OPENROUTER_API_KEY,
)

In [3]:
from langchain_core.prompts import ChatPromptTemplate

FWO_PROMPT = ChatPromptTemplate.from_messages([
    ("system",
     "Du bist der FWO-Assistent (Fachschaft Wirtschaft Olten, FHNW). "
     "Bevor du schreibst, beziehst du dich IMMER auf: "
     "1) Kampagnenkonzept (Ziel, Kernbotschaft, Mehrwert) "
     "2) Redaktionsplan (Phase, wiederkehrende Formate, Timing). "
     "Vermeide Genderung und gebrauche bspw. Lehrperson anstatt Lehrer*innen "
     "Generiere jeweils immer zwei Vorschläge zu jeder Plattform (Instagram, WhatsApp, LinkedIn)"
     "Vermeide Floskeln, schreibe aktiv und konkret."
     "If you are unsure or the answer isn't in the context, say that you don't know.\n\n"
    ),
    ("user",
     "User: {text}")
])


In [4]:
# from langchain_core.output_parsers import StrOutputParser

user_prompt="Generiere mir einen Post für die PubTour am 16. Oktober"

# chain_simple = (FWO_PROMPT | llm | StrOutputParser())
# result = chain_simple.invoke(user_prompt)
# print(result)

In [5]:
from langchain_community.document_loaders import PyPDFLoader

# List of PDF file paths

pdf_path = "Order pdf/SoMe Konzept.pdf"

loader = PyPDFLoader(str(pdf_path))
pages = loader.load()


# Load all documents
# all_pages_pdf = []
# for pdf in pdf_files:
#     loader = PyPDFLoader(pdf)
#     pages = loader.load()
#     all_pages_pdf.extend(pages)

#print(f"Loaded {len(all_pages_pdf)} pages from {len(pdf_files)} pdf documents.")


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# get both websites and pdfs together
all_docs = pages

# define the splitter and strategy
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
splits = splitter.split_documents(all_docs)

In [7]:
import numpy as np

lengths = [len(s.page_content) for s in splits]
print(f"Initial documents: {len(all_docs)}")
print(f"Total chunks: {len(splits)}")
print(f"Avg length: {np.mean(lengths):.1f}")
print(f"Min: {np.min(lengths)}, Max: {np.max(lengths)}")

Initial documents: 6
Total chunks: 55
Avg length: 253.0
Min: 48, Max: 297


In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


C:\Users\amuel\AppData\Local\Temp\ipykernel_26140\4293025327.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [9]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
    normalize_L2=True
)

vector_store.add_documents(documents=splits)

['7243b00a-bdf0-4ea8-b689-43797338d9e7',
 'ff11b571-354f-479a-a3b3-73deb2b2d980',
 'c9a966f4-8b3c-4a86-89fe-6b2e0a380dc9',
 '44a3a4fb-058d-4545-9cae-db2d56cba617',
 '972c28e0-8b7e-4d37-a482-004ce498f4fb',
 '4b7ed638-c79b-4d7f-91a2-078722bb0736',
 '492ddfd4-8d34-48e1-84a6-a13ce01167c0',
 '01128f78-4702-44aa-a792-9d1aa2d02f5a',
 'e450ae02-a8da-48b4-96c4-3d23fc41fbb9',
 'dee28694-91c9-4781-8e98-1c3a99e486c3',
 '3348f234-1066-4540-bebe-11104d0334e0',
 'c031b08e-0522-415e-975c-4ad19f42ae5f',
 'd9d3d565-5629-4653-af83-92ccf1cbf24e',
 '8db6c282-6232-40da-8b45-77f9338a83e8',
 '2bfde551-4946-4d65-bcf4-53071604ab72',
 'e4fa302a-12c4-47af-8230-685c42163f02',
 'b3edcca2-9e40-4634-b30e-2cb2ce36857d',
 '8955e847-d703-474a-a571-e4b6deee3254',
 '0bdc844f-5364-4170-abc7-08a2713d6e29',
 '8857d25f-f7a0-4519-a6d9-96f84899b98b',
 'fb1b128c-ab34-4183-a355-80696515f7f3',
 '2c390f75-cb92-4a7e-84f9-978eac2abf39',
 '006de712-0a81-49bd-8ca2-bd1d898eb22f',
 '0b2807c3-b48c-4238-80b2-8dd1c770683c',
 'b26becc8-81ea-

In [10]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [11]:
docs = retriever.get_relevant_documents("Hashtags für Instagram")
for i, d in enumerate(docs, 1):
    print(f"--- Retrieved doc {i} ---")
    print(d.page_content[:400], "...\n")

--- Retrieved doc 1 ---
- Grund: Account aktiv halten vor allem in Zeiten, bei denen weniger Posts gemacht 
werden können. 
Hashtags: 
#FHNW #FWO #FachschaftWirtschaftOlten #HappyEaster #HappyChristmas #HappyXmas und 
Spezifische Hashtags die von Partnern vorgegeben werden. 
Standard Seeding: ...

--- Retrieved doc 2 ---
Hashtags: Keine Hashtags 
Seeding: kein Seeding. ...

--- Retrieved doc 3 ---
Die meisten Jugendlichen/jungen Erwachsene sind auf Instagram aktiv. Das Posten von Events 
auf diesem Kanal ermöglicht der FWO ein gutes und interessantes Image für die FHNW in Olten 
zu erschaffen und somit zukünftige Studierende zu „überzeugen“ an der FH in Olten studieren zu ...



C:\Users\amuel\AppData\Local\Temp\ipykernel_26140\3188004710.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("Hashtags für Instagram")


In [12]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

# chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | FWO_PROMPT
#     | llm
#     | output_parser
# )

chain = (
    {
        "context": retriever,     # <- Docs -> Text
        "question": RunnablePassthrough(),      # <- nimmt den String-Input direkt
    }
    | FWO_PROMPT
    | llm
    | StrOutputParser()
)


In [13]:
# from langchain_core.output_parsers import StrOutputParser

# USER_PROMPT="Generiere mir einen Post für die PubTour am 16. Oktober"

# result = chain.invoke("Generiere mir einen Post für die PubTour am 16. Oktober")
# print(result)

result = chain.invoke({"question": "Generiere mir einen Post für die PubTour am 16. Oktober", "text": "Generiere mir einen Post für die PubTour am 16. Oktober"})
print(result)

#result = chain.invoke(user_prompt)
print(result)

AttributeError: 'dict' object has no attribute 'replace'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=56a0a349-7f2e-43e5-8d52-5ded467f6e9c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>